In [1]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-
import tensorflow as tf
import numpy
writer = tf.python_io.TFRecordWriter('test1111.tfrecord')
for i in range(0, 2):
    a = 0.618 + i
    b = [2016 + i, 2017+i]
    #c = numpy.array([[0, 1, 2],[3, 4, 5]]) + i
    #c = c.astype(numpy.uint8)
    c = "你好哦"+str(i)
    #c_raw = c.tostring()#这里是把ｃ换了一种格式存储
    c_raw = c
    print 'i:',i
    print '  a:',a
    print '  b:',b
    print '  c:',c
    example = tf.train.Example(
        features = tf.train.Features(
            feature = {'a':tf.train.Feature(float_list = tf.train.FloatList(value=[a])),
                       'b':tf.train.Feature(int64_list = tf.train.Int64List(value = b)),
                       'c':tf.train.Feature(bytes_list = tf.train.BytesList(value = [c_raw]))}))
    serialized = example.SerializeToString()
    writer.write(serialized)
    print '   writer',i,'DOWN!'
writer.close()


i: 0
  a: 0.618
  b: [2016, 2017]
  c: 你好哦0
   writer 0 DOWN!
i: 1
  a: 1.618
  b: [2017, 2018]
  c: 你好哦1
   writer 1 DOWN!


In [2]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-
# output file name string to a queue
filename_queue = tf.train.string_input_producer(['test1111.tfrecord'], num_epochs=None)
# create a reader from file queue
reader = tf.TFRecordReader()
_, serialized_example = reader.read(filename_queue)
# get feature from serialized example


features = tf.parse_single_example(serialized_example,
        features={
            'a': tf.FixedLenFeature([], tf.float32),
            'b': tf.FixedLenFeature([2], tf.int64)
            
        }
    )
a_out = features['a']
b_out = features['b']
#c_out = features['c']
#c_raw_out = features['c']
#c_raw_out = tf.sparse_to_dense(features['c'])
#c_out = tf.decode_raw(c_raw_out, tf.uint8)
print( a_out)
print (b_out)
#print c_out
#c_out = tf.reshape(c_out, [2, 3])
a_batch, b_batch = tf.train.shuffle_batch([a_out, b_out], batch_size=3,
                                capacity=200, min_after_dequeue=100, num_threads=2)
sess = tf.Session()
init = tf.initialize_all_variables()
sess.run(init)

tf.train.start_queue_runners(sess=sess)
a_val, b_val = sess.run([a_batch, b_batch])
# print(a_val, b_val, c_val)
print('first batch:') 
print('  a_val:',a_val)
print('  b_val:',b_val) 

Tensor("ParseSingleExample/ParseSingleExample:0", shape=(), dtype=float32)
Tensor("ParseSingleExample/ParseSingleExample:1", shape=(2,), dtype=int64)
Instructions for updating:
Use `tf.global_variables_initializer` instead.
first batch:
('  a_val:', array([1.618, 1.618, 0.618], dtype=float32))
('  b_val:', array([[2017, 2018],
       [2017, 2018],
       [2016, 2017]]))


In [4]:
!python -V

Python 2.7.15 :: Anaconda, Inc.


In [5]:

import sys
print(sys.executable)

/anaconda2/bin/python
